In [79]:
%pip install -q sql av pillow pandas sqlalchemy ipython-sql pymysql roboflow git+https://github.com/THU-MIG/yolov10.git supervision huggingface_hub bottleneck==1.3.6 numexpr==2.8.4 wget gcloud google google-cloud-storage

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install httplib2==0.15.0 google-api-python-client==1.6

  Using cached google_api_python_client-1.6.0-py2.py3-none-any.whl.metadata (1.1 kB)
Using cached google_api_python_client-1.6.0-py2.py3-none-any.whl (52 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.7.8
    Uninstalling google-api-python-client-1.7.8:
      Successfully uninstalled google-api-python-client-1.7.8

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import pymysql
from pymysql.err import IntegrityError, OperationalError
import os
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av
from PIL import Image
import shutil as sh
import re
from roboflow import Roboflow
import cv2
from ultralytics import YOLOv10
import wget
from time import time
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os


In [64]:
temp_folder = os.path.join('.', 'Files_local')
temp_weights = os.path.join(temp_folder, 'Weights')

In [9]:
bba = sv.BoxCornerAnnotator()
la = sv.LabelAnnotator(text_scale = 0.4, text_padding = 1)

def annotate_image(input_img, model, label_annotator = la, bounding_box_annotator = bba, verbose = False, conf = True, conf_level = 0.05):
    
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model(cont_img, conf=conf_level, verbose = verbose)[0]
    conf_array = np.array(results.boxes.conf.cpu())
    conf_ls_str  = [str(round(x * 100) ) + '%' for x in conf_array]
    tot_time = sum([x for x in results.speed.values()])
    
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    num_oysters = detections.xyxy.shape[0]
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels = conf_ls_str)
    return(annotated_image, num_oysters, tot_time, detections)


def annotate_video(input_vid, model, out_location = '.', im_width = 416, im_height = 416, conf_level = 0.3):
    tot_oysters = 0
    tot_frame = 0
    
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()

    container = av.open(input_vid)
    stream_vid = container.streams.video[0]
    fname = input_vid.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = os.path.join(out_location, f'{fname[:per_index]}_annotated.mp4')
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    start = time()
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg, num_oysters, _, _2z = annotate_image(np_image_2, model, conf_level = conf_level)
        tot_oysters += num_oysters
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    end = time()
    net_time = end - start
    container.close()
    outp.close()
    ann_rate = (index / fps) / net_time # ratio of time to annotate versus length of video
    return tot_oysters / index, net_time, out_path, ann_rate

In [15]:
def get_ext(fpath):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return ext

def get_id_fname(f_out, fpath, id):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return os.path.join(f_out, f"{fname[:per_index]}-{id}.{ext}")

def get_REPLACE_ID(column_id = 'Raw_File_ID', table='raw_files', column_rep='Filepath'):
    cur.execute(f"""SELECT {column_id} from {table} where {column_rep} = 'REPLACE'""")
    id = cur.fetchall()[-1][column_id]
    return id

def get_match(pattern, string):
    matches = re.search(pattern, string)
    if matches:
        print(matches.group(1))
        return(matches.group(1))
    else:
        raise ValueError(f'Improper string exported, couldn\'t find {pattern} in the given text')



def delete_folder(name, base = "."):
    fpath = os.path.join(base, name)
    if os.path.exists(fpath):
        for item in os.listdir(fpath):
            new_path = os.path.join(fpath, item)
            if os.path.isfile(new_path):
                os.remove(new_path)
            elif os.path.isdir(new_path):
                delete_folder(item, fpath)
        os.rmdir(fpath)
                
def make_folder(fpath, overwrite = False):
    delete_folder(fpath, '.') if overwrite else None
    if os.path.exists(fpath):
        
        print(f"{fpath} already exists")
    else:
        os.mkdir(fpath)
        
def get_temp_fname(fpath, temp_f = temp_folder):
    fname = fpath.rsplit('/', 1)[-1]
    return os.path.join(temp_folder, fname)

def trim_file_depth(fpath, start = 0, end = 'Unused'):
    split_ls = fpath.rsplit('/')
    if end == 'Unused':
        end = len(split_ls)
    return '/'.join(split_ls[start:end])

In [199]:
def sign_in_storage_g(JSON_file = 'molten-album-427115-q6-cfa4e4aaf3bd.json'):
    f = open(JSON_file)
    credentials_dict = json.load(f)
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(credentials_dict)
    client = storage.Client(credentials=credentials, project='molten-album-427115-q6')
    bkt = client.get_bucket('test_bucket_abc123')
    return bkt, client

b, cl = sign_in_storage_g()

def check_folder_exists_g(folder_to_upload, check_location, disable = False, full = False, bkt = b): # assume full installation
    # assumes if one file is in the right location then they all are
    try:
        if disable:
            return True
        cur_path = folder_to_upload
        path_check_exist_g = check_location
        if not os.path.exists(cur_path):
            return False
        all_items = os.listdir(cur_path)
        for added_item in all_items: # searching for a file
            cur_path = os.path.join(cur_path, added_item)
            path_check_exist_g = os.path.join(path_check_exist_g, added_item)
            cur_exists = False
            if os.path.isfile(cur_path): # if you just want to make sure one file is the same, set full to 
                cur_exists = path_exists_g(path_check_exist_g)
                if not full: 
                    return cur_exists
            elif os.path.isdir(cur_path):
                cur_exists = check_folder_exists_g(cur_path, path_check_exist_g)
            if not cur_exists:
                return False
        return True
    except Exception:
        print(f"Current folder {folder_to_upload} or {check_location} is malformed")
        return False

def upload_file_g(file_to_upload, new_name_with_path_g, bkt = b):
    blob = bkt.blob(new_name_with_path_g)
    blob.upload_from_filename(file_to_upload)
    

def upload_folder_g(folder_to_upload, new_name_with_path_g, bkt = b , level = 0, overwrite = False):
    already_uploaded = False
    if level == 0: # check if already uploaded
        already_uploaded = check_folder_exists_g(folder_to_upload, new_name_with_path_g)
        level += 1
    if os.path.exists(folder_to_upload) and (not already_uploaded or overwrite):
        for item in os.listdir(folder_to_upload):
            cur_path = os.path.join(folder_to_upload, item)
            new_path_g = os.path.join(new_name_with_path_g, item)
            if os.path.isfile(cur_path):
                upload_file_g(cur_path, new_path_g)
            elif os.path.isdir(new_path_g):
                upload_folder_g(cur_path, new_path_g, level)

def download_file_g(file_to_download_g, new_name_with_path, bkt = b):
    if not os.path.exists(new_name_with_path):
        blob = bkt.blob(file_to_download_g)
        blob.download_to_filename(new_name_with_path)

def list_blobs_g(folder_base, bkt = b, client = cl):
    """Lists all the blobs in the bucket that begin with the prefix."""
    blobs = bkt.list_blobs(prefix=folder_base)
    return blobs

def download_folder_g(folder_to_download_g, new_name_with_path, overwrite = False, level = 0, file_depth = 3):
    already_uploaded = False
    if level == 0: # check if already downloaded
        already_uploaded = os.path.exists(new_name_with_path)
        level += 1
    if not check_folder_exists_g(new_name_with_path, folder_to_download_g) and (not already_uploaded or overwrite):
        for item in list_blobs_g(folder_to_download_g):
            file_path_g = item.name
            file_name = trim_file_depth(file_path_g, start = file_depth)
            local_path = os.path.join(new_name_with_path, file_name)
            download_file_g(file_path_g, local_path)             


def make_folder_g(folder_name, bkt = b):
    blob = bkt.blob(folder_name + '/')
    blob.upload_from_string('')

def path_exists_g(file, bkt = b):
    blob = bkt.blob(file)
    return blob.exists()

In [55]:
upload_file_g("/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Oyster Test Videos/GOPR0018.MP4", "GOPR1077-23.mp4")

In [16]:
if not path_exists_g('Files/'):
    make_folder_g('Files')
    make_folder_g('Files/Video_raw')
    make_folder_g('Files/Video_ann')
    make_folder_g('Files/Image_raw')
    make_folder_g('Files/Image_ann')
    make_folder_g('Files/Model')
    make_folder_g('Files/Roboflow')
make_folder(temp_folder)

./Files_local already exists


In [17]:
db_main = 'test_4'
cur_name = 'ab'


In [228]:
sql_conn = pymysql.connect(
    user="root",
    password="dbuserdbuser",
    host="localhost",
    port=3306,
    database=db_main,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True)

cur = sql_conn.cursor()
res = cur.execute("SELECT 1")
res = cur.fetchall()
res

[{'1': 1}]

In [19]:
def add_user(name, password):
    '''
    Returns: name if query is successful, 0 if not
    '''

    try:
        cur.execute(f"INSERT INTO people (Username, Password, Time_Created) VALUES ('{name}', '{password}', CURRENT_TIMESTAMP );")
    except IntegrityError:
        print(f"Data Integrity Violated: Duplicate User with {name}")
    return name
add_user(cur_name, '123')

Data Integrity Violated: Duplicate User with ab


'ab'

In [218]:
def add_photo(name, fpath, notes = '', f_out = 'Files/Image_raw'):
    '''
    Returns: Index of added photo if successful, 0 if not
    '''

    im = Image.open(fpath)
    width = im.size[0]
    height = im.size[1]
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)
    f_temp = get_temp_fname(fpath)
    im.save(f_temp)
    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Local_Path, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', '{f_temp}', {fsize}, 'Image', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()

    f_id_name_g = get_id_fname(f_out, fpath, id)
    temp_fname = get_temp_fname(f_id_name_g)
    print(temp_fname)
#     print(f_temp, f_id_name)
    upload_file_g(fpath, f_id_name_g)
    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name_g}', Local_Path = '{temp_fname}' WHERE Raw_File_ID = {id};")
    return id

SOME_IMAGE_PATH = "/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Jupyter_local_code/problem_photo.jpg"

id_raw_photo = add_photo(cur_name, fpath = SOME_IMAGE_PATH, notes = 'from HPC')

./Files_local/problem_photo-35.jpg


In [234]:
import cv2
def add_video(name, fpath, notes = '', f_out = 'Files/Video_raw'):
    '''
    Returns: Index of added video if successful, 0 if not
    '''
    cap = cv2.VideoCapture(fpath)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)
    
    f_temp = get_temp_fname(fpath)
    
    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Local_Path, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', 'REPLACE', {fsize}, 'Video', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name_g = get_id_fname(f_out, fpath, id)
    temp_path = get_temp_fname(f_id_name_g)
#     print(temp_path)
    
#     print(fpath, f_id_name_g)
    upload_file_g(fpath, f_id_name_g)
    
    sh.copyfile(fpath, f_id_name_g)

    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name_g}', Local_Path = '{temp_path}' WHERE Raw_File_ID = {id};")

    fps = cap.get(cv2.CAP_PROP_FPS)
    color_order = 'RGB' # FIXME - cant figure out how to extract from cv2 object
    
    

    cur.execute(f"INSERT INTO videos (Raw_File_ID, FPS, Color_Order) VALUES ('{id}', '{fps}', '{color_order}');")

    return id

SOME_VIDEO_PATH = "/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Jupyter_local_code/GOPR1077_tr.mp4"
id_raw_video = add_video(cur_name, SOME_VIDEO_PATH, notes='on new computer')

./Files_local/GOPR1077_tr-37.mp4
/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Jupyter_local_code/GOPR1077_tr.mp4 Files/Video_raw/GOPR1077_tr-37.mp4


In [165]:
def download_weight(path, ver): # one of ['n', 's', 'm', 'b', 'x', 'l']
    valid_ver = ['n', 's', 'm', 'b', 'x', 'l']
    if ver not in valid_ver:
        assert InvalidVersionError('Invalid version selected. Must be one of: n, s, m, l, b, x,  for nano, small, medium, large, big extra big respectively')
    prefix = "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10"
    suffix = ".pt"
    web_path = prefix + ver + suffix
    computer_path = os.path.join(path, 'yolov10' + ver + suffix)
    if not os.path.exists(computer_path):
        wget.download(web_path, out = path)
    return computer_path

def get_weights(f_weights_g = 'Files/Weights'): # TODO - create weights table
    temp_weights = os.path.join(temp_folder, "Weights")
    make_folder(temp_weights)
    prefix = "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10"
    versions = ['n', 's', 'm', 'b', 'x', 'l']
    suffix = ".pt"
    for ver in versions:
        cur_file = 'yolov10' + ver + suffix
        web_path = prefix + ver + suffix
        computer_path = os.path.join(temp_weights, cur_file)
        if not os.path.exists(computer_path):
            wget.download(web_path, out = temp_weights)
    
    upload_folder_g(temp_weights, f_weights_g)
    
get_weights()

./Files_local/Weights already exists


In [159]:


def add_roboflow(name, export_string, f_out = 'Files/Roboflow', f_weights = "Files/Weights"):
    
    '''
    Returns: Index of added roboflow if successful, 0 if not
    '''
    pattern_api = r'api_key\s*=\s*"([^"]+)"'
    api_key_lab = get_match(pattern_api, export_string)

    pattern_workspace = r'rf\.workspace\("([^"]+)"\)'
    workspace_lab = get_match(pattern_workspace, export_string)

    pattern_project = r'project\("([^"]+)"\)'
    project_lab = get_match(pattern_project, export_string)

    pattern_version = r'version\((\d+)\)'
    version_lab = get_match(pattern_version, export_string)

    pattern_download = r'\bdownload\("([^"]+)"\)'
    download_lab = get_match(pattern_download, export_string)
    
#     f_temp = get_temp_fname(f_out)
    folder_name = f'{workspace_lab}_{project_lab}_{version_lab}_{download_lab}'
    f_temp = os.path.join(temp_folder, folder_name)
    
    folder_g = os.path.join(f_out, folder_name)
#     folder_roboflow = f"{f_out}/{workspace_lab}_{project_lab}_{version_lab}_{download_lab}"
    print(api_key_lab, workspace_lab, project_lab, version_lab, download_lab, f_temp)
    download_roboflow(api_key_lab, workspace_lab, project_lab, version_lab, download_lab, f_temp)
    
    upload_folder_g(f_temp, folder_g)
    
    cur.execute(f"INSERT INTO roboflow (Api_Key, Workspace, Project, Version, Download, Dataset_Location, Local_Path, Username, Timestamp) VALUES ('REPLACE', '{workspace_lab}', '{project_lab}', '{version_lab}', '{download_lab}', '{f_temp}', '{f_temp}', '{name}', CURRENT_TIMESTAMP);")
    
    id = get_REPLACE_ID('Roboflow_ID', 'roboflow', 'Api_Key')
    
    cur.execute(f"UPDATE roboflow SET Api_Key = '{api_key_lab}' WHERE Roboflow_ID = {id};")
    
    return id
    
id_rob = add_roboflow(cur_name, \
"""
rf = Roboflow(api_key="Rvk9Clz4ote58KBvCfU3")
project = rf.workspace("curvy-oysters").project("oysters-cjt7n")
dataset = project.version(19).download("yolov8")

""" )




Rvk9Clz4ote58KBvCfU3
curvy-oysters
oysters-cjt7n
19
yolov8
Rvk9Clz4ote58KBvCfU3 curvy-oysters oysters-cjt7n 19 yolov8 ./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`


['']

In [148]:

# download_folder_g("Files/Roboflow/curvy-oysters_oysters-cjt7n_19_yolov8", "./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8")

./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/test/images
Director

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/images
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

...

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/train/labels
Directory {cut_path} already exists
./Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/tra

KeyboardInterrupt: 

In [ ]:
def download_roboflow(api_key, workspace, project, version, download, location):
    # if not os.path.exists(folder_roboflow):
    rf = Roboflow(api_key = api_key)
    project = rf.workspace(workspace).project(project)
    version = project.version(version)
    dat = version.download(download, location = location)
    
    fpath = os.path.join(location, 'data.yaml')
    
    with open(fpath) as f1:
        lines = f1.readlines()
    with open(fpath, 'w') as f2:
        f2.writelines(lines[:-4])
        f2.write("test: ../test/images\ntrain: ../train/images\nval: ../valid/images\n\n")

In [170]:
# upload_file_g('./runs/detect/train2/weights/best.pt', f'./Files/Model/{2000}.pt')

In [182]:
def add_model(roboflow_ID, f_out = "Files/Model", size_mod = 'n', weights_path_g = "Files/Weights", epochs = 10, batch = 32):
    
    weights_path = download_weight(temp_folder, size_mod)
    
    
    cur.execute(f"SELECT * FROM roboflow WHERE Roboflow_ID = {roboflow_ID}")
    res = cur.fetchall()[-1]
    print(res)

    download_roboflow(res['Api_Key'], res['Workspace'], res['Project'], res['Version'], res['Download'], res['Dataset_Location'])
    
    
#     dataset_loc_g = res['Dataset_Location']
    
#     dataset_temp = get_temp_fname(dataset_loc_g)
    
#     print(dataset_loc_g, dataset_temp)
    
#     download_folder_g(dataset_loc_g, dataset_temp)
    
    pt = os.path.join(os.getcwd(), res['Dataset_Location'][len('./'):], 'data.yaml')

    !yolo task=detect mode=train epochs={epochs} batch={batch} plots=False model={weights_path} data={pt}
    
    samp_photo = os.path.join(res['Dataset_Location'], 'test', 'images')
    first_photo = os.listdir(samp_photo)[0]
    im = Image.open(os.path.join(samp_photo, first_photo))
    width = im.size[0]
    height = im.size[1]
                                                               
    cur.execute(f"""INSERT INTO models (Timestamp_Created, Model_Points_Path, Local_Path, Version, 
                 Hyperparams, Epoch, Batch, Model_Type, Width_Training_Images, Height_Training_Images, 
                 Size, Roboflow_ID) values (CURRENT_TIMESTAMP, 'REPLACE', 'REPLACE', 10, NULL, {epochs}, {batch}, 
                 'YOLO', {width}, {height}, '{size_mod}', {roboflow_ID})
                 """)
    
    id_mod = get_REPLACE_ID(column_id = 'Model_ID', table='models', column_rep='Model_Points_Path')
    pts_name = f'{id_mod}.pt'
    model_path_g = os.path.join(f_out, pts_name)
    pts_path = './runs/detect/train/weights/best.pt'
    temp_save_path = os.path.join(temp_folder, pts_name)
    upload_file_g(pts_path, model_path_g)
    os.rename(pts_path, temp_save_path) # for later inference
    
    cur.execute(f"UPDATE models SET Model_Points_Path = '{model_path_g}', Local_Path = '{temp_save_path}' WHERE Model_ID = {id_mod};")

    delete_folder('runs')
    return id_mod

id_mod = add_model(id_rob,epochs=1, size_mod = 'n')
    

{'Roboflow_ID': 19, 'Api_Key': 'Rvk9Clz4ote58KBvCfU3', 'Workspace': 'curvy-oysters', 'Project': 'oysters-cjt7n', 'Version': 19, 'Download': 'yolov8', 'Username': 'ab', 'Timestamp': datetime.datetime(2024, 6, 25, 20, 28, 54), 'Dataset_Location': './Files_local/curvy-oysters_oysters-cjt7n_19_yolov8', 'Local_Path': './Files_local/curvy-oysters_oysters-cjt7n_19_yolov8'}
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`
New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.7 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=./Files_local/yolov10n.pt, data=/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/sql/Files_local/curvy-oysters_oysters-cjt7n_19_yolov8/data

                 Class     Images  Instances      Box(P          R      mAP50  m
                   all         92       1501      0.387      0.258      0.247      0.109
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.0ms postprocess per image
💡 Learn more at https://docs.ultralytics.com/modes/train


In [ ]:
ann_photo_id = 0
id_vid = 0

In [233]:
def ann_img(Raw_File_ID, Model_ID, threshold = 0.3, notes = '', f_out = 'Files/Image_ann'):
    cur.execute(f"SELECT * FROM raw_files WHERE Raw_File_ID = {Raw_File_ID}")
    cur_photo = cur.fetchall()[-1]

    cur.execute(f"SELECT * FROM models WHERE Model_ID = {Model_ID}")
    cur_model = cur.fetchall()[-1]

    cur.execute(f"SELECT * from annotated_files WHERE Model_ID = {Model_ID} AND Raw_File_ID = {Raw_File_ID}")
    res = cur.fetchall()
    if res:
        print('Image Already Annotated')
        return res[-1]['Ann_File_ID']
    
    download_file_g(cur_photo['Filepath'], cur_photo['Local_Path'])
    im = Image.open(cur_photo['Local_Path'])
    np_img = np.array(im)

    download_file_g(cur_model['Model_Points_Path'], cur_model['Local_Path'])
    model = YOLOv10(cur_model['Local_Path'])
    annot, num_oysters, tot_time, end_ann_data = annotate_image(np_img, model, conf_level = threshold)

    cur.execute(f"INSERT INTO annotated_files (Raw_File_ID, Model_ID, Annotated_Filepath, Time_to_Annotate, Confidence_Threshold, Notes, Timestamp) VALUES ('{Raw_File_ID}', '{Model_ID}', 'REPLACE', '{tot_time}', {threshold}, '{notes}', CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID(column_id = 'Ann_File_ID', table='annotated_files', column_rep='Annotated_Filepath')
    f_id_name_g = get_id_fname(f_out, cur_photo['Filepath'], id)
    f_local = get_temp_fname(f_id_name_g)
    print(f_local)

    im_f = Image.fromarray(annot)
    im_f.save(f_local)



    upload_file_g(f_local, f_id_name_g)


    cur.execute(f"UPDATE annotated_files SET Annotated_Filepath = '{f_id_name_g}', Local_Path = '{f_local}' WHERE Ann_File_ID = {id};")
    cur.execute(f"INSERT INTO annotated_photos (Ann_File_ID, Number_of_Oysters) VALUES ({id}, {num_oysters})")

    coord = end_ann_data.xyxy
    conf = end_ann_data.confidence
    names = end_ann_data.data['class_name']
    class_num = end_ann_data.class_id
    for idx in range(len(coord)):
        coord_cur = coord[idx]
        cur.execute(f"INSERT INTO oysters_in_photo (Ann_File_ID, Confidence, X1, Y1, X2, Y2, Class, Class_Index) VALUES ({id}, {conf[idx]}, {coord_cur[0]}, {coord_cur[1]}, {coord_cur[2]}, {coord_cur[3]}, '{names[idx]}', {class_num[idx]});")
    return id
#     except IntegrityError:
#         print("An exception occurred")
#         print(IntegrityError)
#         cur.execute(f"SELECT Ann_File_ID FROM annotated_files WHERE Raw_File_ID = {Raw_File_ID} AND Model_ID = {Model_ID}")
#         return(cur.fetchall()[-1]['Ann_File_ID'])

# cur.execute(f"DELETE FROM oysters_in_photo WHERE Ann_File_ID = {ann_photo_id};")  
# cur.execute(f"DELETE FROM annotated_photos WHERE Ann_File_ID = {ann_photo_id};")
# cur.execute(f"DELETE FROM annotated_files WHERE Ann_File_ID = {ann_photo_id};")

ann_photo_id = ann_img(id_raw_photo, id_mod)

./Files_local/problem_photo-35-57.jpg


In [259]:
def ann_video(Raw_File_ID, Model_ID, notes = '', f_out = 'Files/Video_ann', threshold = 0.3):
    cur.execute(f"SELECT * FROM raw_files WHERE Raw_File_ID = {Raw_File_ID}")
    cur_video = cur.fetchall()[-1]

    cur.execute(f"SELECT * FROM models WHERE Model_ID = {Model_ID}")
    cur_model = cur.fetchall()[-1]

    cur.execute(f"SELECT * from annotated_files WHERE Model_ID = {Model_ID} AND Raw_File_ID = {Raw_File_ID}")
    res = cur.fetchall()
    if res:
        return res[-1]['Ann_File_ID']
    
    download_file_g(cur_model['Model_Points_Path'], cur_model['Local_Path'])
    
    model = YOLOv10(cur_model['Local_Path'])

    download_file_g(cur_video['Filepath'], cur_video['Local_Path'])
    
    avg_oysters, time_s, out_path, ann_rate = annotate_video(cur_video['Local_Path'], model, out_location = temp_folder, conf_level = threshold)

    cur.execute(f"INSERT INTO annotated_files (Raw_File_ID, Model_ID, Annotated_Filepath, Time_to_Annotate, Confidence_Threshold, Notes, Timestamp) VALUES ('{Raw_File_ID}', '{Model_ID}', 'REPLACE', '{time_s * 1000}', {threshold}, '{notes}', CURRENT_TIMESTAMP);")

    id = get_REPLACE_ID(column_id = 'Ann_File_ID', table='annotated_files', column_rep='Annotated_Filepath')

    f_id_name_g = get_id_fname(f_out, cur_video['Filepath'], id)
    
    f_local = get_temp_fname(f_id_name_g)
#     print(f_local, out_path, f_id_name_g)
    
    cur.execute(f"UPDATE annotated_files SET Annotated_Filepath = '{f_id_name_g}', Local_Path = '{f_local}' WHERE Ann_File_ID = {id};")

    os.rename(out_path, f_local)

    cur.execute(f"INSERT INTO annotated_videos (Ann_File_ID, Annotation_Rate, Tracing, Average_Number_of_Oysters) VALUES ({id}, {ann_rate}, 0, {avg_oysters})")

    upload_file_g(f_local, f_id_name_g)
    
    return id

# cur.execute(f"DELETE FROM oysters_in_photo WHERE Ann_File_ID = {id_vid};")  
# cur.execute(f"DELETE FROM annotated_videos WHERE Ann_File_ID = {id_vid};")
# cur.execute(f"DELETE FROM annotated_files WHERE Ann_File_ID = {id_vid};")

id_vid = ann_video(id_raw_video, id_mod)